In [1]:
# 🔍 Extraction PDF et découpage du texte
from pypdf import PdfReader

def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    return " ".join(page.extract_text() or "" for page in reader.pages)

def split_text(text, max_length=500):
    sentences = text.split(". ")
    chunks, chunk = [], ""
    for sentence in sentences:
        if len(chunk) + len(sentence) < max_length:
            chunk += sentence + ". "
        else:
            chunks.append(chunk.strip())
            chunk = sentence + ". "
    if chunk:
        chunks.append(chunk.strip())
    return chunks

def clean_chunk(chunk):
    return chunk.replace("\xa0", " ").replace("\n", " ").strip()


In [2]:
# 📄 Lecture du PDF et préparation des chunks
pdf_path = "data/loups_garous.pdf"
raw_text = extract_text_from_pdf(pdf_path)
chunks = split_text(raw_text)
print(f"✅ {len(chunks)} morceaux extraits.")


✅ 40 morceaux extraits.


In [3]:
# 🔡 Embedding des textes + FAISS
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
corpus_embeddings = embedder.encode(chunks, convert_to_numpy=True)

index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(corpus_embeddings)


c:\Users\laeti\OneDrive\Documents\Formation Dev IA - GRETA\Nin\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 🤖 Chargement du modèle Mistral-7B-Instruct-v0.2
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16,
    device_map="auto"
)


Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [5]:
import gradio as gr

def generate_response(question):
    q_embedding = embedder.encode([question], convert_to_numpy=True)
    D, I = index.search(q_embedding, k=5)
    
    top_chunks = [clean_chunk(chunks[i]) for i in I[0]]
    context = " ".join(top_chunks)
    
    prompt = f"""
Tu es un expert des règles de jeux de société. Ci-dessous se trouve un extrait du manuel de règles.

Contexte extrait du manuel :
{context}

Question posée par un joueur :
{question}

Réponds précisément et clairement en te basant uniquement sur ce contexte. Si la réponse n’est pas dans le texte, indique-le poliment. Sois pédagogique et clair.
"""
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            num_beams=1
        )
    
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    used_chunks = "\n\n".join(f"[Chunk {i+1}]\n{chunks[i]}" for i in I[0])
    
    return response.strip(), used_chunks

# 🔧 Interface Gradio adaptée
iface = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, placeholder="Exemple : Que se passe-t-il si deux joueurs ont le même nombre de votes ?", label="Question sur une règle du jeu"),
    outputs=[
        gr.Textbox(label="Réponse claire à votre question"),
        gr.Textbox(label="Extraits du manuel utilisés (pour transparence)")
    ],
    title="Assistant Règles - Jeu de Société",
    description="Posez une question sur une règle que vous ne comprenez pas, et l'assistant vous explique simplement à partir du manuel."
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
